In [1]:
import pandas as pd
import numpy as np
import itertools
from scipy.optimize import fmin_tnc

In [49]:
data = pd.read_csv('data/rawdata/2015.txt', sep = ';')

/Users/adamlevin/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (13,20,21,22,27,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [50]:
data.columns = np.array([str(i).strip() for i in list(data.columns.values)]) #remove space in col names

In [51]:
data.columns

Index([u'Tour Code', u'Tour Description', u'Year', u'Tourn.#', u'Player #',
       u'Course #', u'Permanent Tournament #', u'Player First Name',
       u'Player Last Name', u'Round', u'Tournament Name', u'Course Name',
       u'Hole', u'Hole Score', u'Par Value', u'Yardage', u'Shot',
       u'Shot Type(S/P/D)', u'# of Strokes', u'From Location(Scorer)',
       u'From Location(Enhanced)', u'To Location(Laser)',
       u'To Location(Enhanced)', u'Distance', u'Distance to Pin',
       u'In the Hole Flag', u'Around the Green Flag', u'1st Putt Flag',
       u'Distance to Hole after the Shot', u'Time', u'Lie', u'Elevation',
       u'Slope', u'X Coordinate', u'Y Coordinate', u'Z Coordinate',
       u'Distance from Center', u'Distance from Edge', u'Date'],
      dtype='object')

In [52]:
data['X Coordinate'] = [str(i).replace(' ','') for i in data['X Coordinate']] #remove space in coordinates cols
data['Y Coordinate'] = [str(i).replace(' ','') for i in data['Y Coordinate']]
data['Z Coordinate'] = [str(i).replace(' ','') for i in data['Z Coordinate']]

In [53]:
data['X Coordinate'] = [str(i).replace(',','') for i in data['X Coordinate']] #remove commas in coordinates cols
data['Y Coordinate'] = [str(i).replace(',','') for i in data['Y Coordinate']]
data['Z Coordinate'] = [str(i).replace(',','') for i in data['Z Coordinate']]

In [54]:
data['X Coordinate'] = ['-' + str(i)[:len(str(i))-1] if str(i)[len(str(i))-1]=='-' else i for i in data['X Coordinate']] #putting negative in front
data['Y Coordinate'] = ['-' + str(i)[:len(str(i))-1] if str(i)[len(str(i))-1]=='-' else i for i in data['Y Coordinate']]
data['Z Coordinate'] = ['-' + str(i)[:len(str(i))-1] if str(i)[len(str(i))-1]=='-' else i for i in data['Z Coordinate']]

In [55]:
data['X Coordinate'] = pd.to_numeric(data['X Coordinate'])
data['Y Coordinate'] = pd.to_numeric(data['Y Coordinate'])
data['Z Coordinate'] = pd.to_numeric(data['Z Coordinate'])

In [6]:
#unique Course-Round-Hole Tuples
uCRHtps = list(itertools.product(np.unique(data['Course Name']),np.unique(data['Round']),np.unique(data['Hole'])))

In [58]:
# coordinates of hole are not given. must be imputed.
# does the distance use the x,y, and z coordinates or just the x and y coordinates?
# test: first find the hole using the x,y, and z coordinates and record the average difference between calculated hole
# location and ball and recorded distance. then do the same using just the x and y coodinates. see which is better.

In [59]:
# def f (a):
#     x0,y0,z0 = a[0],a[1],a[2]
#     return sum((((x-x0)**2 + (y-y0)**2 + (z-z0)**2)**.5-d)**2)/len(x)

# def find_the_hole ():
#     xopt = fmin_tnc(f,[x0,y0,z0],approx_grad=1)[0].tolist()
#     return xopt

In [60]:
# #finding the coordinates of the hole
# aveds=[]
# for u,i in enumerate(uCRHtps):
# #     if (u+1)%100==0:
# #         print u
# #         print u+1,sum(aveds)/len(aveds)
#     subset = data[(data['Course Name']==i[0]) & (data['Round']==int(i[1])) & (data['Hole']==int(i[2])) \
#                   & (data['Distance to Hole after the Shot']!=0) & (data['X Coordinate']!=0) & (data['Y Coordinate']!=0) \
#                   & (data['Z Coordinate']!=0)]
#     if subset.shape[0] == 0:
#         continue
#     d = subset['Distance to Hole after the Shot'].values/12.0
#     x = subset['X Coordinate'].values
#     y = subset['Y Coordinate'].values
#     z = subset['Z Coordinate'].values
#     sorted_subset = subset.sort_values('Distance to Hole after the Shot')
#     x0 = sorted_subset['X Coordinate'].values[0] ##assume that closest ball recorded to hole does not have an error
#     y0 = sorted_subset['Y Coordinate'].values[0]
#     z0 = sorted_subset['Z Coordinate'].values[0]
#     subset.insert(len(subset.columns),'approx_dist',((x-x0)**2 + (y-y0)**2 + (z-z0)**2)**.5)
#     # dist_diff is difference between recorded distance and distance approximated from location of closest recorded shot
#     subset.insert(len(subset.columns),'dist_diff', \
#                   np.absolute(subset['approx_dist'].values - subset['Distance to Hole after the Shot'].values/12.0))
#     before = subset.shape[0]
#     # remove very inconsistant shots from record which are likely mistakes
#     subset = subset[subset['dist_diff']<sorted_subset['Distance to Hole after the Shot'].values[0]]
#     after = subset.shape[0]
#     subset.drop('approx_dist',axis=1,inplace=True)
#     subset.drop('dist_diff',axis=1,inplace=True)
#     if before-after>0:
#         print u, before-after
#     d = subset['Distance to Hole after the Shot'].values/12.0
#     x = subset['X Coordinate'].values
#     y = subset['Y Coordinate'].values
#     z = subset['Z Coordinate'].values
#     a = find_the_hole()
#     subset.insert(len(subset.columns),'dist_w_impute',np.array(((x-a[0])**2 + (y-a[1])**2 + (z-a[2])**2)**.5).tolist())
#     subset.insert(len(subset.columns),'dist_diff', \
#                   np.absolute(subset['dist_w_impute'].values - subset['Distance to Hole after the Shot'].values/12.0))
#     mean_err = subset['dist_diff'].mean()
#     if mean_err>.27:
#         print u, mean_err
#     max_err = subset['dist_diff'].max()
#     if max_err>.5:
#         print u,max_err
    
# #print sum(aveds)/len(aveds)

In [61]:
# def f (a):
#     x0,y0 = a[0],a[1]
#     return sum((((x-x0)**2 + (y-y0)**2)**.5-d)**2)/len(x)

# def find_the_hole ():
#     xopt = fmin_tnc(f,[x0,y0],approx_grad=1)[0].tolist()
#     return xopt

In [62]:
# #finding the coordinates of the hole
# aveds=[]
# for u,i in enumerate(uCRHtps):
# #     if (u+1)%100==0:
# #         print u
# #         print u+1,sum(aveds)/len(aveds)
#     subset = data[(data['Course Name']==i[0]) & (data['Round']==int(i[1])) & (data['Hole']==int(i[2])) \
#                   & (data['Distance to Hole after the Shot']!=0) & (data['X Coordinate']!=0) & (data['Y Coordinate']!=0) \
#                   & (data['Z Coordinate']!=0)]
#     if subset.shape[0] == 0:
#         continue
#     d = subset['Distance to Hole after the Shot'].values/12.0
#     x = subset['X Coordinate'].values
#     y = subset['Y Coordinate'].values
#     sorted_subset = subset.sort_values('Distance to Hole after the Shot')
#     x0 = sorted_subset['X Coordinate'].values[0] ##assume that closest ball recorded to hole does not have an error
#     y0 = sorted_subset['Y Coordinate'].values[0]
#     subset.insert(len(subset.columns),'approx_dist',((x-x0)**2 + (y-y0)**2)**.5)
#     # dist_diff is difference between recorded distance and distance approximated from location of closest recorded shot
#     subset.insert(len(subset.columns),'dist_diff', \
#                   np.absolute(subset['approx_dist'].values - subset['Distance to Hole after the Shot'].values/12.0))
#     before = subset.shape[0]
#     # remove very inconsistant shots from record which are likely mistakes
#     subset = subset[subset['dist_diff']<sorted_subset['Distance to Hole after the Shot'].values[0]]
#     after = subset.shape[0]
#     subset.drop('approx_dist',axis=1,inplace=True)
#     subset.drop('dist_diff',axis=1,inplace=True)
#     if before-after>0:
#         print u, before-after
#     d = subset['Distance to Hole after the Shot'].values/12.0
#     x = subset['X Coordinate'].values
#     y = subset['Y Coordinate'].values
#     z = subset['Z Coordinate'].values
#     a = find_the_hole()
#     subset.insert(len(subset.columns),'dist_w_impute',np.array(((x-a[0])**2 + (y-a[1])**2)**.5).tolist())
#     subset.insert(len(subset.columns),'dist_diff', \
#                   np.absolute(subset['dist_w_impute'].values - subset['Distance to Hole after the Shot'].values/12.0))
#     mean_err = subset['dist_diff'].mean()
#     if mean_err>.27:
#         print u, mean_err
#     max_err = subset['dist_diff'].max()
#     if max_err>.5:
#         print u,max_err
    
# #print sum(aveds)/len(aveds)

In [63]:
# based on results of number of points that have implausible coordinates, and average error between the recorded dist.
# and this dist. calculated with the imputed coordinates of the hole, it is clear that the recorded distances use 
# the x,y, and z coordinates.
# now recreate dataframe with concatonation of all subsets - without bunk records and with the x,y, and z coordinates
# of the hole.

In [69]:
def f (a):
    x0,y0,z0 = a[0],a[1],a[2]
    return sum((((x-x0)**2 + (y-y0)**2 + (z-z0)**2)**.5-d)**2)/len(x)

def find_the_hole ():
    xopt = fmin_tnc(f,[x0,y0,z0],approx_grad=1)[0].tolist()
    return xopt

In [78]:
# initializing new data frame with two rows which will be deleted after
newdata = pd.DataFrame(data.loc[1:2,:])

# finding the coordinates of the hole and recording the results
# must treat shots that were holed out differently since the coordinates are recorded as 0 for those shots
for u,i in enumerate(uCRHtps):
    if (u)%100==1:
        print 'it = ', u
    subset = data[(data['Course Name']==i[0]) & (data['Round']==int(i[1])) & (data['Hole']==int(i[2])) \
                  & (~((data['Distance to Hole after the Shot']!=0) & \
                       ((data['X Coordinate']==0) | (data['Y Coordinate']==0) | (data['Z Coordinate']==0))))]
    if subset[subset['Distance to Hole after the Shot']!=0].shape[0] == 0:
        continue
    start = subset.shape[0]
    subset = subset.sort_values('Distance to Hole after the Shot')
    d = subset[subset['Distance to Hole after the Shot']!=0]['Distance to Hole after the Shot'].values/12.0
    x = subset[subset['Distance to Hole after the Shot']!=0]['X Coordinate'].values
    y = subset[subset['Distance to Hole after the Shot']!=0]['Y Coordinate'].values
    z = subset[subset['Distance to Hole after the Shot']!=0]['Z Coordinate'].values
    x0 = subset[subset['Distance to Hole after the Shot']!=0]['X Coordinate'].values[0] ##assume that closest ball recorded to hole does not have an error
    y0 = subset[subset['Distance to Hole after the Shot']!=0]['Y Coordinate'].values[0]
    z0 = subset[subset['Distance to Hole after the Shot']!=0]['Z Coordinate'].values[0]
    subset.insert(len(subset.columns),'approx_dist',np.array([0]*subset[subset['Distance to Hole after the Shot']==0].shape[0] \
                  + (((x-x0)**2 + (y-y0)**2 + (z-z0)**2)**.5).tolist()))
    # dist_diff is difference between recorded distance and distance approximated from location of closest recorded shot
    subset.insert(len(subset.columns),'dist_diff', \
                  np.absolute(subset['approx_dist'].values - subset['Distance to Hole after the Shot'].values/12.0))
    # remove very inconsistant shots from record which are likely mistakes
    subset = subset[subset['dist_diff']<subset[subset['Distance to Hole after the Shot']!=0]['Distance to Hole after the Shot'].values[0]]
    finish = subset.shape[0]
    if start-finish>5:
        print u, start-finish
    d = subset[subset['Distance to Hole after the Shot']!=0]['Distance to Hole after the Shot'].values/12.0
    x = subset[subset['Distance to Hole after the Shot']!=0]['X Coordinate'].values
    y = subset[subset['Distance to Hole after the Shot']!=0]['Y Coordinate'].values
    z = subset[subset['Distance to Hole after the Shot']!=0]['Z Coordinate'].values
    a = find_the_hole()
    subset.drop('approx_dist',axis=1,inplace=True)
    subset.drop('dist_diff',axis=1,inplace=True)
    subset.insert(len(subset.columns),'Hole X Coordinate',np.array([a[0]]*subset.shape[0]))
    subset.insert(len(subset.columns),'Hole Y Coordinate',np.array([a[1]]*subset.shape[0]))
    subset.insert(len(subset.columns),'Hole Z Coordinate',np.array([a[2]]*subset.shape[0]))
    subset.insert(len(subset.columns),'dist_w_impute',np.array([0]*subset[subset['Distance to Hole after the Shot']==0].shape[0] \
                  + (((x-a[0])**2 + (y-a[1])**2 + (z-a[2])**2)**.5).tolist()))
    subset.insert(len(subset.columns),'dist_diff', \
                  np.absolute(subset['dist_w_impute'].values - subset['Distance to Hole after the Shot'].values/12.0))
    mean_err = subset[subset['Distance to Hole after the Shot']!=0]['dist_diff'].mean()
    if mean_err>.27:
        print u, 'mean_err = ', mean_err
    if mean_err>1:
        continue
    max_err = subset['dist_diff'].max()
    if max_err>.5:
        print u, 'max_err = ', max_err
    subset.drop('dist_w_impute',axis=1,inplace=True)
    subset.drop('dist_diff',axis=1,inplace=True)
    newdata = newdata.append(subset)

newdata.drop(newdata.head(2).index, inplace=True)
print newdata.shape

it =  1
it =  101
it =  201
it =  301
it =  401
it =  501
it =  601
it =  701
it =  801
808 max_err =  1.50825469801
it =  901
it =  1001
it =  1101
it =  1201
it =  1301
it =  1401
it =  1501
it =  1601
it =  1701
it =  1801
it =  1901
it =  2001
it =  2101
it =  2201
2258 mean_err =  2.8873006469
it =  2301
it =  2401
it =  2501
it =  2601
it =  2701
it =  2801
it =  2901
it =  3001
it =  3101
it =  3201
it =  3301
(1042069, 42)


In [71]:
# pd.options.display.max_rows = 2421
# pd.reset_option("^display")


In [79]:
print data.shape
print newdata.shape
print (data.shape[0]-newdata.shape[0])/float(data.shape[0])

# data has been shrunk by about 10 % as a result of a couple of screening processes:
# I removed all shots with a x,y, or z coordinate equaling 0 but a nonzero recorded distance
# I removed all shots from course-hole-round subsets without any shots that had nonzero recorded distances
# I removed all shots that had recored distances that were not consistant with the coordinates recorded
#
# *details* I did the last one by comparing the absolute value of the difference between distance calculated
# with coordinates of the closest recorded shot and difference recorded and the distance of the closest shot
# recorded. If the distance recorded and the coordinates recorded are plausible, the former must be less than
# the latter. If this was not true, it indicates some sort of error in the recording of the coordinates so these
# shots were discarded.

(1176101, 39)
(1042069, 42)
0.113963001477


In [80]:
#newdata.to_csv('data/2015_with_hole_coordinates.csv')

In [9]:
#newdata.isnull().sum()

In [3]:
data = pd.read_csv('data/2014_with_hole_coordinates.csv', sep = ',')

In [4]:
#data[['Course Name','Player First Name','Round','Hole','Hole Score','Shot']]
data.columns

Index([u'Unnamed: 0', u'# of Strokes', u'1st Putt Flag',
       u'Around the Green Flag', u'Course #', u'Course Name', u'Date',
       u'Distance', u'Distance from Center', u'Distance from Edge',
       u'Distance to Hole after the Shot', u'Distance to Pin', u'Elevation',
       u'From Location(Enhanced)', u'From Location(Scorer)', u'Hole',
       u'Hole Score', u'Hole X Coordinate', u'Hole Y Coordinate',
       u'Hole Z Coordinate', u'In the Hole Flag', u'Lie', u'Par Value',
       u'Permanent Tournament #', u'Player #', u'Player First Name',
       u'Player Last Name', u'Round', u'Shot', u'Shot Type(S/P/D)', u'Slope',
       u'Time', u'To Location(Enhanced)', u'To Location(Laser)', u'Tour Code',
       u'Tour Description', u'Tourn.#', u'Tournament Name', u'X Coordinate',
       u'Y Coordinate', u'Yardage', u'Year', u'Z Coordinate'],
      dtype='object')

In [ ]:
# goal is to impute tee box locations. will use all shots with 'Shot' column equal to 1
# will do the same process of optimizing a guess as when imputing the location of the hole.
# will use a randomly selected shot as initial guess, then run optimization, then filter out
# shots over a certian threshold of implausibility, then rereun the optimization.

In [5]:
def f (a):
    x0,y0,z0 = a[0],a[1],a[2]
    return sum((((x-x0)**2 + (y-y0)**2 + (z-z0)**2)**.5-d)**2)/len(x)

def find_the_tee_box ():
    xopt = fmin_tnc(f,[x0,y0,z0],approx_grad=1,disp=1)[0].tolist()
    return xopt

In [9]:
# initializing new data frame with two rows which will be deleted after
newdata = pd.DataFrame(data.loc[1:2,:])

for u,i in enumerate(uCRHtps):
    if u%333==0:
        print u, newdata.shape[0]
    subset = data[(data['Course Name']==i[0]) & (data['Round']==int(i[1])) & (data['Hole']==int(i[2]))]
    before = subset.shape[0]
    if subset[subset['Distance to Hole after the Shot']!=0].shape[0] == 0:
        continue
    d = subset[subset['Shot']==1]['Distance'].values/12.0
    x = subset[subset['Shot']==1]['X Coordinate'].values
    y = subset[subset['Shot']==1]['Y Coordinate'].values
    z = subset[subset['Shot']==1]['Z Coordinate'].values
    rand_ind = np.random.choice(range(subset[subset['Shot']==1].shape[0]),size=1)
    rand_shot = subset[subset['Shot']==1][['X Coordinate','Y Coordinate','Z Coordinate']].values[rand_ind,:].tolist()[0]
    x0,y0,z0 = rand_shot[0],rand_shot[1],rand_shot[2]
    a = find_the_tee_box()
    subset.insert(len(subset.columns),'dist_w_impute',np.array([0]*subset[subset['Shot']!=1].shape[0] \
                  + (((x-a[0])**2 + (y-a[1])**2 + (z-a[2])**2)**.5).tolist()))
    subset.insert(len(subset.columns),'dist_diff',np.array([abs(subset['dist_w_impute'].values[u] - \
                  subset['Distance'].values[u]/12) if subset['Shot'].values[u]==1 else 0 for u in range(subset.shape[0])]))
    mean_err = subset[subset['dist_diff']>0]['dist_diff'].mean()
    std_err = subset[subset['dist_diff']>0]['dist_diff'].std()
    while mean_err>180:
        subset.drop(subset[subset['dist_diff'] > mean_err + 3*std_err].index,axis=0,inplace=True)
        subset.drop('dist_w_impute',axis=1,inplace=True)
        subset.drop('dist_diff',axis=1,inplace=True)
        d = subset[subset['Shot']==1]['Distance'].values/12.0
        x = subset[subset['Shot']==1]['X Coordinate'].values
        y = subset[subset['Shot']==1]['Y Coordinate'].values
        z = subset[subset['Shot']==1]['Z Coordinate'].values
        rand_ind = np.random.choice(range(subset[subset['Shot']==1].shape[0]),size=1)
        rand_shot = subset[subset['Shot']==1][['X Coordinate','Y Coordinate','Z Coordinate']].values[rand_ind,:].tolist()[0]
        x0,y0,z0 = rand_shot[0],rand_shot[1],rand_shot[2]
        a = find_the_tee_box()
        subset.insert(len(subset.columns),'dist_w_impute',np.array([0]*subset[subset['Shot']!=1].shape[0] \
                  + (((x-a[0])**2 + (y-a[1])**2 + (z-a[2])**2)**.5).tolist()))
        subset.insert(len(subset.columns),'dist_diff',np.array([abs(subset['dist_w_impute'].values[u] - \
                  subset['Distance'].values[u]/12) if subset['Shot'].values[u]==1 else 0 for u in range(subset.shape[0])]))
        mean_err = subset[subset['dist_diff']>0]['dist_diff'].mean()
        std_err = subset[subset['dist_diff']>0]['dist_diff'].std()
    subset.drop('dist_w_impute',axis=1,inplace=True)
    subset.drop('dist_diff',axis=1,inplace=True)
    subset.insert(len(subset.columns),'Tee Box X Coordinate',np.array([a[0]]*subset.shape[0]))
    subset.insert(len(subset.columns),'Tee Box Y Coordinate',np.array([a[1]]*subset.shape[0]))
    subset.insert(len(subset.columns),'Tee Box Z Coordinate',np.array([a[2]]*subset.shape[0]))
    after = subset.shape[0]
    if before-after>0:
        print u, before-after
    newdata = newdata.append(subset)

newdata.drop(newdata.head(2).index, inplace=True)
print newdata.shape


0 2


/Users/adamlevin/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/adamlevin/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/adamlevin/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/adamlevin/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A val

238 1


KeyboardInterrupt: 